<a href="https://colab.research.google.com/github/MariaZharova/HM-recommender-system/blob/main/H%26M_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Data structure research

In [11]:
articles = pd.read_csv('articles.csv')
customers = pd.read_csv('customers.csv')
ratings = pd.read_csv('transactions_train.csv')

In [3]:
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1.0,Ladieswear,16.0,Womens Everyday Basics,1002.0,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1.0,Ladieswear,16.0,Womens Everyday Basics,1002.0,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1.0,Ladieswear,16.0,Womens Everyday Basics,1002.0,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1.0,Ladieswear,61.0,Womens Lingerie,1017.0,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1.0,Ladieswear,61.0,Womens Lingerie,1017.0,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [4]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22590 entries, 0 to 22589
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   article_id                    22590 non-null  int64  
 1   product_code                  22590 non-null  int64  
 2   prod_name                     22590 non-null  object 
 3   product_type_no               22590 non-null  int64  
 4   product_type_name             22590 non-null  object 
 5   product_group_name            22590 non-null  object 
 6   graphical_appearance_no       22590 non-null  int64  
 7   graphical_appearance_name     22590 non-null  object 
 8   colour_group_code             22590 non-null  int64  
 9   colour_group_name             22590 non-null  object 
 10  perceived_colour_value_id     22590 non-null  int64  
 11  perceived_colour_value_name   22589 non-null  object 
 12  perceived_colour_master_id    22589 non-null  float64
 13  p

In [9]:
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [10]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34735 entries, 0 to 34734
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             34735 non-null  object 
 1   FN                      11957 non-null  float64
 2   Active                  11669 non-null  float64
 3   club_member_status      34561 non-null  object 
 4   fashion_news_frequency  34315 non-null  object 
 5   age                     34328 non-null  float64
 6   postal_code             34735 non-null  object 
dtypes: float64(3), object(4)
memory usage: 1.9+ MB


In [12]:
ratings.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001.0,0.050831,2.0
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023.0,0.030492,2.0
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004.0,0.015237,2.0
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003.0,0.016932,2.0
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004.0,0.016932,2.0


In [13]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1251759 entries, 0 to 1251758
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   t_dat             1251759 non-null  object 
 1   customer_id       1251759 non-null  object 
 2   article_id        1251758 non-null  float64
 3   price             1251758 non-null  float64
 4   sales_channel_id  1251758 non-null  float64
dtypes: float64(3), object(2)
memory usage: 47.8+ MB


## Data preprocessing

In [20]:
# Merge datasets
ratings_df = pd.merge(ratings, customers, on='customer_id')
ratings_df = pd.merge(ratings_df, articles, on='article_id')
# Separate the features
x = ratings_df[['customer_id', 'article_id']]
x_train, x_test = train_test_split(x, test_size=0.3, random_state=10)

## Implementation of ASVD algorithm

In [ ]:
def ASVD_reccommender(train_global_rating_mean, user, item, f, train_record, test_record, train_rating, test_rating, alpha):
    ASVD_u = train_global_rating_mean
    ASVD_bu = tf.Variable(tf.zeros((user, 1), dtype=tf.float32))
    ASVD_bi = tf.Variable(tf.zeros((1, item), dtype=tf.float32))
    ASVD_Q = tf.Variable(tf.random_normal([item, f], stddev=1/(f**0.5), dtype=tf.float32))
    ASVD_x = tf.Variable(tf.random_normal([item,f], stddev=1/(f**0.5), dtype=tf.float32))
    ASVD_y = tf.Variable(tf.random_normal([item,f], stddev=1/(f**0.5), dtype=tf.float32))
    ASVD_train_record = tf.constant(train_record, dtype=tf.float32)
    ASVD_test_record = tf.constant(test_record, dtype=tf.float32)
    N = np.zeros((user, item), dtype=np.float32)
    R = np.zeros((user, item), dtype=np.float32)
    ASVD_Nu = np.sum(train_record, axis=1)

    for i in range(0, user):
        if ASVD_Nu[i] == 0:
            ASVD_Nu[i] = 1
    for i in range(0, item):
        N[:, i] = ASVD_Nu

    ASVD_Nu = N ** 0.5
    ASVD_Ru = np.sum(test_record, axis=1)

    for i in range(user):
        if ASVD_Ru[i] == 0:
            ASVD_Ru[i] = 1
    for i in range(item):
        R[:,i] = ASVD_Ru

    ASVD_Ru = R ** 0.5
    ASVD_QX = tf.matmul(ASVD_Q, ASVD_x, transpose_b=True)
    ASVD_Qxt = tf.matmul(ASVD_test_record, ASVD_QX)
    ASVD_Qxt =(test_rating - ASVD_u - ASVD_bi - ASVD_bu) * ASVD_Qxt
    ASVD_QY = tf.matmul(ASVD_Q, ASVD_y, transpose_b=True)
    ASVD_Qyt = tf.matmul(ASVD_train_record, ASVD_QY)
    ASVD_predict_rating = ASVD_u + ASVD_bu + ASVD_bi + ASVD_Qxt/ASVD_Ru + ASVD_Qyt / ASVD_Nu
    ASVD_loss = tf.reduce_sum(((train_rating-ASVD_predict_rating)*train_record)**2) + alpha * \
                             ((tf.reduce_sum(ASVD_x) ** 2) + (tf.reduce_sum(ASVD_y) ** 2) + \
                              (tf.reduce_sum(ASVD_bu) ** 2) + (tf.reduce_sum(ASVD_bi) ** 2) + \
                              (tf.reduce_sum(ASVD_Q) ** 2))
                             
    return ASVD_loss, ASVD_predict_rating